In [1]:
import pandas as pd 
import numpy as np
import requests
import json
import time
import datetime as dt
import re
import os.path
from openpyxl import load_workbook
import os
import warnings
warnings.filterwarnings('ignore')
import codecs

In [2]:
Area_Code = {
    'DZ':['Algeria','DZA'],
    'AO':['Angola','AGO'],
    'BJ':['Benin','BEN'],
    'BW':['Botswana','BWA'],
    'BF':['Burkina Faso','BFA'],
    'BI':['Burundi','BDI'],
    'CV':['Cabo Verde','CPV'],
    'CM':['Cameroon','CMR'],
    'CF':['Central African Republic','CAF'],
    'TD':['Chad','TCD'],
    'KM':['Comoros','COM'],
    'CG':['Congo','COG'],
    'CI':['Côte d’Ivoire','CIV'],
    'CD':['Democratic Republic of the Congo','COD'],
    'DJ':['Djibouti','DJI'],
    'EG':['Egypt','EGY'],
    'GQ':['Equatorial Guinea','GNQ'],
    'ER':['Eritrea','ERI'], #232
    'SZ':['Eswatini','SWZ'],
    'ET':['Ethiopia','ETH'],
    'GA':['Gabon','GAB'],
    'GM':['Gambia','GMB'],
    'GH':['Ghana','GHA'],
    'GN':['Guinea','GIN'],
    'GW':['Guinea-Bissau','GNB'],
    'KE':['Kenya','KEN'],
    'LS':['Lesotho','LSO'],
    'LR':['Liberia','LBR'],
    'LY':['Libya','LBY'],
    'MG':['Madagascar','MDG'],
    'MW':['Malawi','MWI'],
    'ML':['Mali','MLI'],
    'MR':['Mauritania','MRT'],
    'MU':['Mauritius','MUS'],
    'MA':['Morocco','MAR'],
    'MZ':['Mozambique','MOZ'],
    'NA':['Namibia','NAM'],
    'NE':['Niger','NER'],
    'NG':['Nigeria','NGA'],
    'RW':['Rwanda','RWA'],
    'SN':['Senegal','SEN'],
    'SC':['Seychelles','SYC'],
    'SL':['Sierra Leone','SLE'],
    'SO':['Somalia','SOM'],
    'ZA':['South Africa','ZAF'],
    'SS':['South Sudan','SSD'],
    'ST':['So Tom & Prncipe','STP'],
    'SD':['Sudan','SDN'],
    'TG':['Togo','TGO'],
    'TN':['Tunisia ','TUN'],
    'UG':['Uganda','UGA'],
    'TZ':['United Republic of Tanzania','TZA'], #835 834
    'ZM':['Zambia','ZMB'],
    'ZW':['Zimbabwe','ZWE'],
}
# Getting year for citation
D = dt.datetime.today()
y = D.year

In [4]:
Indicators = {
    'BCG_GADAFX_T_G01_XDC':['Net Acquisition of Financial Assets Other than Cash, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADAFM_T_G01_CA_XDC':['Monetary Gold and SDRs, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GR_G01_CA_XDC':['Revenue, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GRT_G01_CA_XDC':['Taxes, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GRS_G01_CA_XDC':['Social Contributions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GRG_G01_CA_XDC':['Grants, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GRO_G01_CA_XDC':['Other Revenue, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GE_G01_CA_XDC':['Expense, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GECE_G01_CA_XDC':['Compensation of Employees, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GEGS_G01_CA_XDC':['Use of Goods and Services, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GEI_G01_CA_XDC':['Interest, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GEST_G01_CA_XDC':['Subsidies and Transfers, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GEG_G01_CA_XDC':['Grants, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GES_G01_CA_XDC':['Social Benefits, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GEO_G01_CA_XDC':['Other Expense, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GXCCO_G01_CA_XDC':['Cash Inflow from Operation Activities, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADAN_T_G01_CA_XDC':['Non-financial Assets, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADANP_T_G01_CA_XDC':['Purchases of Non-financial Assets, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADANS_T_G01_CA_XDC':['Sales of Non-financial Assets, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GXCCB_G01_CA_XDC':['Cash surplus/deficit, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADAFX_T_G01_CA_XDC':['Net Acquisition of Financial Assets Other than Cash, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADAFXD_T_G01_CA_XDC':['Domestic, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADAFXF_T_G01_CA_XDC':['Foreign, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADL_T_G01_CA_XDC':['Liabilities, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLSDR_T_G01_CA_XDC':['Special Drawing Rights (SDRs), Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLCD_T_G01_CA_XDC':['Currency and Deposits, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLSO_T_G01_CA_XDC':['Securities Other than Shares, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLLS_T_G01_CA_XDC':['Loans, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLAE_T_G01_CA_XDC':['Shares and Other Equity, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLIR_T_G01_CA_XDC':['Insurance Technical Reserves, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLFD_T_G01_CA_XDC':['Financial Derivatives, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLD_T_G01_CA_XDC':['Domestic, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GADLF_T_G01_CA_XDC':['Foreign, Transactions, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GXCCF_G01_CA_XDC':['Cash Inflow from Financing Activities, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GXCNC_G01_CA_XDC':['Net change in the stock of cash, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GXCCBA_G01_CA_XDC':['Adjustment to Cash surplus/deficit, Cash (Budg. Cen. Govt.)','Net', 'NCR'],
    'BCG_GX_G01_CA_XDC':['Total','TOTAL','NCR'],
}
              

In [5]:
Data=[]
D_source = 'International Monetary Fund (IMF)'
D_providerID= 2

for num,Ind_code in enumerate(Indicators):

    url = f'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A..{Ind_code}.?startPeriod=2010&endPeriod=2021'
    a = 0
    data = None
    print(f"Indicator {num}:",Ind_code)
    Ind_name = Indicators[Ind_code][0]    
    S_source= f'Retrived from: {url}, On: {str(D.date())}, By web scraping using the python libraries: requests and pandas'
    Mode_code=Indicators[Ind_code][1]
    Unit_code=Indicators[Ind_code][2]
    while data is None:
        try:
            a +=1
            print("\t atempt:",a)
            data = (requests.get(url).json()['CompactData']['DataSet']['Series'])
        except:
            time.sleep(5)
            continue
    for i in range(len(data)):
        if 'Obs' in data[i]:
            obs = data[i]['Obs']
            area_code = data[i].get('@REF_AREA')
            if area_code in Area_Code:
                area_name = Area_Code[area_code][0]
                area_c = Area_Code[area_code][1]
                if type(obs)==list:
                    for o in obs:
                        if o.get('@OBS_VALUE') is not None:
                            Data.append([area_c,area_name,D_source,D_providerID,\
                                         S_source,Ind_code,Ind_name,Mode_code,\
                                         Unit_code,o.get('@TIME_PERIOD'),\
                                         o.get('@OBS_VALUE')])
                else:
                    if o.get('@OBS_VALUE') is not None:
                        Data.append([area_c,area_name,D_source,D_providerID,\
                                     S_source,Ind_code,Ind_name,Mode_code,\
                                     Unit_code,o.get('@TIME_PERIOD'),\
                                     o.get('@OBS_VALUE')])
                        
DataFrame = pd.DataFrame(Data, columns=['Areacode','Areaenglish','DataSource',\
                                        'DataProviderId','SpecificSource',\
                                        'Indicatorcode','Indicatorname','Modecode',\
                                        'Unitcode','Year', 'Value'])
DataFrame = DataFrame.sort_values(by=['Areaenglish'], ascending=True)
DataFrame.to_excel('Gov_Finance.xlsx', index=False)
print('All Done!')


Indicator 0: BCG_GADAFX_T_G01_XDC
	 atempt: 1
Indicator 1: BCG_GADAFM_T_G01_CA_XDC
	 atempt: 1
Indicator 2: BCG_GR_G01_CA_XDC
	 atempt: 1
Indicator 3: BCG_GRT_G01_CA_XDC
	 atempt: 1
Indicator 4: BCG_GRS_G01_CA_XDC
	 atempt: 1
Indicator 5: BCG_GRG_G01_CA_XDC
	 atempt: 1
Indicator 6: BCG_GRO_G01_CA_XDC
	 atempt: 1
Indicator 7: BCG_GE_G01_CA_XDC
	 atempt: 1
Indicator 8: BCG_GECE_G01_CA_XDC
	 atempt: 1
Indicator 9: BCG_GEGS_G01_CA_XDC
	 atempt: 1
Indicator 10: BCG_GEI_G01_CA_XDC
	 atempt: 1
Indicator 11: BCG_GEST_G01_CA_XDC
	 atempt: 1
Indicator 12: BCG_GEG_G01_CA_XDC
	 atempt: 1
Indicator 13: BCG_GES_G01_CA_XDC
	 atempt: 1
Indicator 14: BCG_GEO_G01_CA_XDC
	 atempt: 1
Indicator 15: BCG_GXCCO_G01_CA_XDC
	 atempt: 1
Indicator 16: BCG_GADAN_T_G01_CA_XDC
	 atempt: 1
Indicator 17: BCG_GADANP_T_G01_CA_XDC
	 atempt: 1
Indicator 18: BCG_GADANS_T_G01_CA_XDC
	 atempt: 1
Indicator 19: BCG_GXCCB_G01_CA_XDC
	 atempt: 1
Indicator 20: BCG_GADAFX_T_G01_CA_XDC
	 atempt: 1
Indicator 21: BCG_GADAFXD_T_G01_CA

In [5]:
url = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/CompactData/IFS/A..BCG_GRT_G01_CA_XDC.?startPeriod=2010&endPeriod=2021'

In [6]:
data = (requests.get(url).json()['CompactData']['DataSet']['Series'])

In [7]:
data

[{'@FREQ': 'A',
  '@REF_AREA': 'KW',
  '@INDICATOR': 'BCG_GRT_G01_CA_XDC',
  '@UNIT_MULT': '6',
  '@TIME_FORMAT': 'P1Y',
  'Obs': [{'@TIME_PERIOD': '2010', '@OBS_VALUE': '318.1'},
   {'@TIME_PERIOD': '2011', '@OBS_VALUE': '326.3'},
   {'@TIME_PERIOD': '2012', '@OBS_VALUE': '85.6'},
   {'@TIME_PERIOD': '2013', '@OBS_VALUE': '394.2'},
   {'@TIME_PERIOD': '2014', '@OBS_VALUE': '420'}]},
 {'@FREQ': 'A',
  '@REF_AREA': 'DM',
  '@INDICATOR': 'BCG_GRT_G01_CA_XDC',
  '@UNIT_MULT': '6',
  '@TIME_FORMAT': 'P1Y',
  'Obs': [{'@TIME_PERIOD': '2010', '@OBS_VALUE': '327.912341'},
   {'@TIME_PERIOD': '2011', '@OBS_VALUE': '311.928656'},
   {'@TIME_PERIOD': '2012', '@OBS_VALUE': '302.172517'},
   {'@TIME_PERIOD': '2013', '@OBS_VALUE': '303.292812'},
   {'@TIME_PERIOD': '2014', '@OBS_VALUE': '314.940339'},
   {'@TIME_PERIOD': '2015', '@OBS_VALUE': '329.469886'}]},
 {'@FREQ': 'A',
  '@REF_AREA': 'JP',
  '@INDICATOR': 'BCG_GRT_G01_CA_XDC',
  '@UNIT_MULT': '6',
  '@TIME_FORMAT': 'P1Y',
  'Obs': [{'@TIME_PE

In [8]:
data[0].get('@REF_AREA')

'KW'